### Домашняя работа к уроку 5
### Студент: Абрамов А.В.

Задание 1. Написать теггер на данных с русским языком:

-проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации

-написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

-сравнить все реализованные методы, сделать выводы  


Задание 2. Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities

-проверить NER из nltk/spacy/deeppavlov.

-написать свой NER, попробовать разные подходы.

а) передаём в сетку токен и его соседей.

б) передаём в сетку только токен.

в) свой вариант.

-сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [ ]:
!mkdir datasets
!wget https://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip -d ./datasets

mkdir: cannot create directory ‘datasets’: File exists
--2023-05-04 11:19:26--  https://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip.2’

collection5.zip.2   100%[===================>]   1.81M  7.59MB/s    in 0.2s    

2023-05-04 11:19:27 (7.59 MB/s) - ‘collection5.zip.2’ saved [1899530/1899530]

Archive:  collection5.zip
replace ./datasets/Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import nltk
import os
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import spacy
from spacy import displacy
import en_core_web_md
import ru_core_news_lg

import corus
from corus import load_ne5
from razdel import tokenize

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:895: UserWarning: [W094] Model 'en_core_web_md' (2.2.0) specifies an under-constrained spaCy version requirement: >=2.2.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.5.2,<3.6.0
  warnings.warn(warn_msg)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data..

True

In [ ]:
dataset_for_nltk = []

for filename in os.scandir("datasets/Collection5/"):
    if filename.is_file():
        name, ext = os.path.splitext(filename.path)
        if ext == ".txt":
          with(open(filename.path,"r",encoding='utf-8') as f):
            text = f.read()
            tokens = text.split()
          
          tagged_tokens = {}

          with(open(name + ".ann","r",encoding='utf-8') as f):
            for line in f:
              parts = line.split()
              tag = parts[1]
              name = ' '.join(parts[4:])
              tagged_tokens[name] = tag

          dataset_for_nltk.append((text, tagged_tokens))


In [ ]:
for entry in dataset_for_nltk:
  pred = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(entry[0]))) if hasattr(chunk, 'label')]
  print(entry[1])
  print(pred)
  break

# Работает довольно плохо

{'БРЯНСК': 'LOC', 'РИА Новости': 'MEDIA', 'Анна Сенина': 'PER', 'Андрей Пономарев': 'PER', 'Николая Денина': 'PER', 'Брянской области': 'LOC', 'Дмитрий Огнев': 'PER', 'Пономарев': 'PER', 'Иван Тимохин': 'PER', 'Брянской областной думой': 'ORG', 'Огнев': 'PER', 'Яблоко': 'ORG', 'ЛДПР': 'ORG', 'Михаилом Марченко': 'PER', 'Денина': 'PER', 'Вадима Потомского': 'PER', 'Пономарева': 'PER', 'Денин': 'PER', 'Верховном суде': 'ORG', 'Марченко': 'PER', 'Совете Федерации': 'ORG', 'Людмилу Нарусову': 'PER', 'Александр Горшков': 'PER', 'Лариса Погорелова': 'PER', 'Рашит Габдулвалеев': 'PER'}
[('БРЯНСК', 'GPE'), ('Анна Сенина', 'PERSON'), ('Андрей Пономарев', 'PERSON'), ('Николая Денина', 'PERSON'), ('Дмитрий Огнев', 'PERSON'), ('Пономарев', 'PERSON'), ('Кроме', 'PERSON'), ('Пономарев', 'PERSON'), ('Таким', 'PERSON'), ('Вадима Потомского', 'PERSON'), ('Денин', 'PERSON'), ('Марченко', 'ORGANIZATION'), ('Совете Федерации', 'PERSON'), ('Людмилу Нарусову', 'PERSON'), ('Горшков', 'PERSON'), ('Лариса Пого

In [ ]:
nlp = ru_core_news_lg.load()
doc = nlp(dataset_for_nltk[0][0])
displacy.render(doc, jupyter=True, style='ent')

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!locale

LANG=en_US.UTF-8
LANGUAGE=
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE="en_US.UTF-8"
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=


In [ ]:
def load_text_patched(path):
    # do not convert \r\n to \n
    with open(path, newline='', encoding='utf-8') as file:
        return file.read()

dir = 'datasets/Collection5/'
# load_ne5 do not accept encoding, but Colab sometimes fails to default to utf-8 for unknown reasons
corus.ne5.load_text = load_text_patched
records = load_ne5(dir)

In [ ]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

datasets/Collection5/26_11_12e.txt
datasets/Collection5/1152.txt
datasets/Collection5/1107.txt
datasets/Collection5/004.txt
datasets/Collection5/443.txt
datasets/Collection5/kamchatka.txt
datasets/Collection5/last_54.txt
datasets/Collection5/114.txt
datasets/Collection5/242.txt
datasets/Collection5/25_12_12d.txt
datasets/Collection5/627.txt
datasets/Collection5/021.txt
datasets/Collection5/057.txt
datasets/Collection5/1182.txt
datasets/Collection5/1146.txt
datasets/Collection5/457.txt
datasets/Collection5/396.txt
datasets/Collection5/28_11_12f.txt
datasets/Collection5/272.txt
datasets/Collection5/last_03.txt
datasets/Collection5/118.txt
datasets/Collection5/002.txt
datasets/Collection5/157.txt
datasets/Collection5/620.txt
datasets/Collection5/last_15.txt
datasets/Collection5/505.txt
datasets/Collection5/048.txt
datasets/Collection5/160.txt
datasets/Collection5/413.txt
datasets/Collection5/428.txt
datasets/Collection5/581.txt
datasets/Collection5/354.txt
datasets/Collection5/611.txt
dat

In [ ]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [ ]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 128

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 5),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [ ]:
embedding_dim = 128

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [ ]:
model = modelNER()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['acc'])

In [ ]:
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 84s 7ms/step - loss: 0.1024 - acc: 0.9664 - val_loss: 0.2535 - val_acc: 0.8962
Epoch 2/5
12444/12444 [==============================] - 82s 7ms/step - loss: 0.0992 - acc: 0.9670 - val_loss: 0.2612 - val_acc: 0.8959
Epoch 3/5
12444/12444 [==============================] - 82s 7ms/step - loss: 0.0976 - acc: 0.9676 - val_loss: 0.2711 - val_acc: 0.8950
Epoch 4/5
12444/12444 [==============================] - 83s 7ms/step - loss: 0.0968 - acc: 0.9678 - val_loss: 0.2835 - val_acc: 0.8948
Epoch 5/5
12444/12444 [==============================] - 82s 7ms/step - loss: 0.0961 - acc: 0.9678 - val_loss: 0.2711 - val_acc: 0.9387


In [ ]:
y_pred1 = model.predict(valid_x)
y_pred = np.argmax(y_pred1, axis=1)

print(f'Precision score {precision_score(valid_y, y_pred , average="macro")}')
print(f'Recall score {recall_score(valid_y, y_pred , average="macro")}')
print(f'F1 score {f1_score(valid_y, y_pred , average="macro")}')

2074/2074 [==============================] - 6s 3ms/step
Precision score 0.9188477989540954
Recall score 0.7682516646226318
F1 score 0.8281895170964603


Одно слово (5 эпох):  
Train Accuracy: 0.9678  
Val Accuracy: 0.9387  
Precision: 0.9188477989540954  
Recall: 0.7682516646226318  
F1: 0.8281895170964603  
  
С (1,5)-грамами (3 эпохи):  
Train Accuracy: 0.9657  
Val Accuracy: 0.9412  
Precision: 0.9226781221438194  
Recall: 0.7843320758632909  
F1: 0.8371973641394906